In [1]:
import json
import os.path
import pathlib
# import bdgd_tools
import time
# from bdgd_tools import Sample, Case, Circuit, LineCode
import geopandas as gpd
import re

class JsonData:
    def __init__(self, file_name):
        """
        Inicializa a classe JsonData com o nome do arquivo de entrada.

        :param file_name: Nome do arquivo JSON de entrada.
        """
        self.data = self._read_json_file(file_name)
        self.tables = self._create_tables()

    @staticmethod
    def _read_json_file(file_name):
        """
        Lê o arquivo JSON fornecido e retorna o conteúdo como um objeto Python.

        :param file_name: Nome do arquivo JSON de entrada.
        :return: Objeto Python contendo o conteúdo do arquivo JSON.
        """
        with open(file_name, 'r', encoding='utf-8') as file:
            data = json.load(file)
        return data

    def _create_tables(self):
        """
        Cria um dicionário de tabelas a partir dos dados carregados do arquivo JSON.

        :return: Dicionário contendo informações das tabelas a serem processadas.
        """
        return {
            table_name: Table(
                table_name,
                table_data["columns"],
                table_data["type"],
                table_data["ignore_geometry"],
            )
            for table_name, table_data in self.data["configuration"][
                "tables"
            ].items()
        }

    def get_tables(self):
        """
        Retorna o dicionário de tabelas.

        :return: Dicionário contendo informações das tabelas a serem processadas.
        """
        return self.tables

    @staticmethod
    def convert_data_types(df, column_types):
        """
        Converte os tipos de dados das colunas do DataFrame fornecido.

        :param df: DataFrame a ser processado.
        :param column_types: Dicionário contendo mapeamento de colunas para tipos de dados.
        :return: DataFrame com tipos de dados convertidos.
        """
        return df.astype(column_types)

    def create_geodataframes(self, file_name, runs=1):
        """
        Cria GeoDataFrames a partir de um arquivo de entrada e coleta estatísticas.

        :param file_name: Nome do arquivo de entrada.
        :param runs: Número de vezes que cada tabela será carregada e convertida (padrão: 1).
        :return: Dicionário contendo GeoDataFrames e estatísticas.
        """
        geodataframes = {}

        for table_name, table in self.tables.items():
            load_times = []
            conversion_times = []
            gdf_converted = None

            for _ in range(runs):
                start_time = time.time()
                gdf_ = gpd.read_file(file_name, layer=table.name, include_fields=table.columns,
                                     ignore_geometry=table.ignore_geometry)
                start_conversion_time = time.time()
                gdf_converted = self.convert_data_types(gdf_, table.data_types)                    
                end_time = time.time()
 
                load_times.append(start_conversion_time - start_time)
                conversion_times.append(end_time - start_conversion_time)

            load_time_avg = sum(load_times) / len(load_times)
            conversion_time_avg = sum(conversion_times) / len(conversion_times)
            mem_usage = gdf_converted.memory_usage(index=True, deep=True).sum() / 1024 ** 2

            geodataframes[table_name] = {
                'gdf': gdf_converted,
                'memory_usage': mem_usage,
                'load_time_avg': load_time_avg,
                'conversion_time_avg': conversion_time_avg,
                'ignore_geometry': table.ignore_geometry
            }
        return geodataframes




In [2]:
import pandas as pd

class Table:
    def __init__(self, name, columns, data_types, ignore_geometry_):
        self.name = name
        self.columns = columns
        self.data_types = data_types
        self.ignore_geometry = ignore_geometry_

    def __str__(self):
        return f"Table(name={self.name}, columns={self.columns}, data_types={self.data_types}, " \
               f"ignore_geometry={self.ignore_geometry})"


In [3]:
json_file_name = os.path.join(os.getcwd(), "bdgd2dss.json")

json_data = JsonData(json_file_name)

In [4]:
geodataframes = json_data.create_geodataframes(r"C:\Users\ADERALDO\Desktop\TCC BDGB\BDGD_Dev\bdgd-tools\bdgd_tools\sample\raw\aneel\muxenergia.gdb")
# geodataframes = json_data.create_geodataframes(r"C:\Users\ADERALDO\Desktop\TCC BDGB\BDGD_Dev\bdgd-tools\bdgd_tools\sample\raw\aneel\muxenergia.gdb")
# geodataframes2 = json_data.create_geodataframes(r"C:\Users\ADERALDO\Desktop\TCC BDGB\ENEL_CE_39_2021-12-31_V10_20230124-0859.gdb")

In [9]:
df = geodataframes["CTMT"]['gdf']['COD_ID'].tolist()
df

['1_TAP2_1', '2_TAP2_1']

In [5]:
nom_alimentador = '2_TAP2_1'
# alimentador = "CTMT=={nom_alimentador}"
df = geodataframes["UCBT_tab"]['gdf']
var = df.query("CTMT==@nom_alimentador")
# df
var

,PAC,PN_CON,CTMT,TIP_CC,FAS_CON,TEN_FORN
2,UC14865,1284,2_TAP2_1,RES_2,BCN,15
3,UC15172,7585,2_TAP2_1,RES_2,AN,10
7,UC15176,1321,2_TAP2_1,COM_13,ABCN,15
10,UC14865,1284,2_TAP2_1,RES_2,CAN,15
12,UC20061,7553,2_TAP2_1,RES_7,ABCN,15
...,...,...,...,...,...,...
12336,UC321,1341,2_TAP2_1,RES_2,CN,10
12339,UC328,164,2_TAP2_1,RES_11,AN,10
12347,UC738,1975,2_TAP2_1,RES_2,CN,10
12350,UC8812,71,2_TAP2_1,RES_2,CN,10


In [23]:
var = "cu"
if not os.path.exists(f"sample/{var}"):
    os.mkdir(f'sample/{var}')

output_directory= os.path.join(os.getcwd(), f'output/{feeder}')


In [25]:
output_directory= os.path.join(os.getcwd(), f'output\{var}')
print(output_directory)


c:\Users\ADERALDO\Desktop\TCC BDGB\BDGD_Dev\bdgd-tools\bdgd_tools\output\cu
